In [1]:
import pandas as pd
import os
import subprocess
from shutil import *
import yaml
from scipy.special import erf
import numpy as np

### Inputs

- `cmd_file`$ \ $: $ \ $ Provide *``.cfile``* name if within the working directory, if not the complete path of the *``.cfile``*
- `paths`$ \ \ \ \ \ \ $ : $ \ $ List with all the folder names
- `init_dir`$ \ $: $ \ $ Working directory, `os.getcwd()` if the script is in the directory with all the simulations


In [2]:
cmd_file='Ribstrains.cfile'
paths=['A','B']
init_dir=os.getcwd()

## Parameters to calculate the risk

In [3]:
#local criteria
alpha=0.3026
b0=-2.9866
b1=-0.0130
age=50

#global criteria
int=3.579805
coef_age=-0.00553
log_scale=-1.47593

#### Chest Local tissue injury criteria
$
Failure\ risk(strain,Age)=\frac{1}{2}+\frac{1}{2}erf\left[ \frac{LN(strain)-(\beta_0+age*\beta_1))}{\alpha*\sqrt{2}} \right]
$

$
Total\ number\ of\ fractured\ ribs = \sum_{i=1}^{24}𝑓𝑟𝑎𝑡𝑢𝑟𝑒\ 𝑟𝑖𝑠𝑘\ 𝑓𝑜𝑟\ 𝑟𝑖𝑏\ 𝑖
$

In [4]:
def fracture_risk_calc(rib_strains,age):
    fracture_risk=0
    for y in range(len(rib_strains)):
        prob=0.5+(0.5*erf((np.log(rib_strains[y][0])-(b0+(b1*age)))/(alpha*np.sqrt(2))))
        fracture_risk=fracture_risk+prob
    return fracture_risk

#### Chest Global injury Criteria
$
risk(\%)=\frac{1}{2}+\frac{1}{2}erf\left[ \frac{ln(injury\ criteria)-(int+age*coef\_age)}{\sqrt{2*(exp(log-scale)^{2})}} \right]
$

In [5]:
def chest_global_risk_calc(injury_criteria,age):
    # risk=0
    risk=(0.5+(0.5*erf((np.log(injury_criteria)-(int+(coef_age*age)))/(np.sqrt(2*(np.exp(log_scale))**2)))))*100
    return risk

## Postprocess using .cfile and store result in yaml file

In [6]:
for key,p in enumerate(paths):
    os.chdir(init_dir)
    copy(cmd_file, p)
    os.chdir(p)

    command_process=subprocess.Popen('C:\Progs\LSTC\LSDYNA\lsprepost4.6_x64.exe {}'.format(cmd_file), stdout=subprocess.PIPE)
    output, err = command_process.communicate() 

    ## Extract the strains and calculate risk probability
    strains={}
    vals=['L','R']
    for k,v in enumerate(vals):
        for i in range(12):
            data=pd.read_csv('{}{}.csv'.format(v,i+1),delimiter=',',header = None, skiprows=2)
            strain=max(data.iloc[: , 1:].max())
            strains['{}{}'.format(v,i+1)]=strain
            os. remove('{}{}.csv'.format(v,i+1))
    df_strain=pd.DataFrame.from_dict(strains, orient='index')
    st=df_strain.values
    # df_strain.to_csv('max_strains.csv')

    ## Write the results yaml file
    res={}
    res['Number of fractured ribs']=float(fracture_risk_calc(st,age))
    
    #extract HIC15
    df_hic=pd.read_csv('hic15.csv',skiprows=8,nrows=1, delimiter=',', header=None)
    res['HIC 15']=float(df_hic[0].str.split('=')[0][1])
    os. remove('hic15.csv')
    os. remove('resultantData')
    #extract chest deflection
    df_chest=pd.read_csv('chest_def.csv', header=None,skiprows=2)
    initial_len=df_chest[df_chest.iloc[:,0]==0].values
    os. remove('chest_def.csv')
    res['chest deflection']=float(((initial_len[0][1]-min(df_chest.iloc[:,1]))/initial_len[0][1])*100)
    res['fracture risk global criteria']=float(chest_global_risk_calc(res['chest deflection'],age))
    with open('result.yaml','w') as sol:
        yaml.dump(res,sol,default_flow_style = False, sort_keys=False)


## Compile to a single dataset

In [7]:
os.chdir(init_dir)
Result_set = pd.DataFrame(data=None,columns=None,dtype=None,copy=False)
for k,v in enumerate(paths):
    os.chdir(v)
    with open('result.yaml','r') as file:
        out = yaml.load(file, Loader=yaml.FullLoader) 
    df_output_set = pd.DataFrame.from_dict(out, orient='index').T
    df_output_set.insert(0, "simulation file", v, True)
    Result_set=Result_set.append(df_output_set,ignore_index=True)
    os. remove('result.yaml')
    os.chdir(init_dir)
Result_set.to_csv("simulation_results.csv", index=False)